#### Bag-o-rama   
This notebook will help you create basic reports, securely copy files, and create BagIt-compliant packages complete with custom tags and profiles. It will also remove hidden macOS system files before hashing. It has been tested with a 64 bit Anaconda environment (featuring Jupyter Notebook) in Windows, but should work in macOS and system Python with minor adjustments. It requires BagIt python (https://github.com/LibraryOfCongress/bagit-python) and Grabbags (https://github.com/amiaopensource/grabbags). Credit for metadata report and robocopy cells to staff at American Folklife Center!

Please read the directions, and run each cell one-at-a-time, in order. To run a cell, click it and press ```ctrl``` and ```enter``` on your keyboard.  

Connect the donor's drive to your PC via a write-blocker. To generate a basic metadata report on what the drive contains, replace ```D:\``` in the cell below with the drive letter of your source directory, then run the cell. Be patient - for a large number of files, this may take several hours. When this step is complete, a .csv containing the report will be saved at ```home\downloads\fileinfo.csv```. If you are working with multiple drives, repeat this process for each but rename ```fileinfo.csv``` in Windows Explorer to avoid overwriting it.

In [ ]:
%%cmd
dir -r D:\ | select Name,Length,@{Name="KB";Expression={[math]::round($_.Length/1KB, 2)}},@{Name="MB";Expression={[math]::round($_.Length/1MB, 2)}},@{Name="GB";Expression={[math]::round($_.Length/1GB, 2)}},CreationTime,LastWriteTime,BaseName,Extension,FullName,PSIsContainer,@{n='MD5';ex={(Get-FileHash -Algorithm MD5 $_.FullName).hash}} | Export-Csv -NoTypeInformation -path $home\Downloads\fileinfo.csv

To securely copy content from the drive, in the next cell replace ```A:\src``` with the path of your source directory and ```B:\dst``` with the path of your destination directory. Then, run the cell. Do not close the window until the process has completed.  Again, this step may take several hours. If you are working with multiple drives, repeat this process for each.

In [ ]:
%%cmd
robocopy A:\src B:\dst /E /COPY:DATS /log+:robocopy /tee

Now, arrange the content you wish to bag in Windows Explorer like so:  

```
\parent directory
    \directory to be bagged
        \items to be bagged
    \directory to be bagged
        \items to be bagged
```
    
If you are creating a single bag, your package will feature one parent directory and one directory to be bagged (or child directory). If you are creating multiple bags, your package will feature one parent directory and one child directory per intended number of bags. The parent directory can be named whatever you like; the child directories must be named in accordance with your intended bag name.  

---  

Note 1: Make sure your child directories contain collection items ONLY. Additional metadata and reports will be added later.  

Note 2: Remove Mac system folders from the top level of your child directories manually before proceeding (these directories will appear as grayed-out in Windows Explorer and their filenames will begin with a period). Right click them, delete, then empty your Recycle Bin.  

---  

Now replace ```X:\\insert\\path\\to\\parent\\directory``` in the next cell with the path to your parent directory (be sure to use double backslashes instead of singles). Then, run the cell and wait. Be patient - this step may take several hours.

In [ ]:
%%cmd
grabbags --no-system-files --md5 --processes 4 X:\\insert\\path\\to\\parent\\directory

Your content will now be rearraanged into BagIt structure, like so:  

```
\bag directory
    \data
```

Now, copy any reports you wish to include as tag files (e.g. ```fileinfo.csv``` from step 1, an Exiftool report, or a bulk_extractor report) to your bag directory. Do not add anything to the data directory. Next, replace ```X:\\insert\\path\\to\\bag\\here``` in the cell below with the path to your bag (be sure to use double backslashes instead of singles). Then, run the cell. If you created multiple bags in the previous operation, repeat this process for every bag. 

In [ ]:
import bagit
import os

# Add reports to tag manifest

# load the bag
bag = bagit.Bag('X:\\insert\\path\\to\\bag\\here')

# update tag metadata
bag.save()

Now, replace ```X:\\insert\\path\\to\\bag\\here``` in the cell below with the path to your bag (be sure to use double backslashes instead of singles). Edit the sample information in the cell to add information you wish to include in the BagInfo.txt file, then run the cell. If you created multiple bags earlier, repeat this process for every bag.

In [ ]:
# Add custom fields to baginfo.txt (American Folklife Center profile used as sample)

# load the bag
bag = bagit.Bag('X:\\insert\\path\\to\\bag\\here')

# enter bag info metadata
bag.info['Profile Name'] = 'AFC-Default'
bag.info['LCCN-Permalink'] = ''
bag.info['LC-Master-Files'] = ''
bag.info['LC-Bag-Id'] = ''
bag.info['Local-Description'] = ''
bag.info['LC-Content-Type'] = 'mixed'
bag.info['Internal-Sender-Indentifier'] = ''
bag.info['LC-LCCN'] = ''
bag.info['LC-Content-Process'] = 'content_transfer'
bag.info['LC-Items'] = ''
bag.info['LC-Project'] = 'afcaccessioning'
bag.info['LC-Content-Provider'] = 'afc'
bag.info['Local-Contact-Email'] = ''
bag.info['Local-Contact-Name'] = ''
bag.info['External Description'] = ''
bag.info['Local-Organization'] = 'American Folklife Center'
bag.info['AFC-Type'] = 'hard disk drive'

# save to baginfo.txt
bag.save()

If all of the above ran without error, your bag(s) are now ready. Congratulations! 